In [1]:
import numpy as np
from collections import deque

import matplotlib.pyplot as plt

import opendssdirect as dss
# load the pre-defined grid to opendss, we use the grid sample in /pycigar/tutorials/data/ieee3.dss
# this grid is really simple and only contains 3 loads for the purpose of demonstration
dss.run_command("Redirect ./data/ieee3.dss")

from pycigar.core.kernel.kernel import Kernel

In [2]:


# get the list of load names to see if the grid is loaded sucessfully
print(dss.Loads.AllNames())

#
k = Kernel(simulator="opendss", sim_params=None)

# get the 4 sub-kernels
k_simulation = k.simulation
k_scenario = k.scenario
k_node = k.node
k_device = k.device
#####################################
# get the kernel from the sub-kernels
#these functions below should return the same kernel object
k1 = k_simulation.master_kernel
k2 = k_scenario.master_kernel
k3 = k_node.master_kernel
k4 = k_device.master_kernel

# assert that we get the same kernel object
assert(k1 == k2 == k3 == k4)

kernel_api = k.simulation.start_simulation()

k_simulation.pass_api(kernel_api)
k_scenario.pass_api(kernel_api)
k_node.pass_api(kernel_api)
k_device.pass_api(kernel_api)

# or just simply call 
k.pass_api(kernel_api)


['s701a', 's702a', 's703a']


In [3]:
# load pre-defined grid .dss to OpenDSS
dir_opendss = "./data/ieee3.dss"
kernel_api.simulation_command("Redirect " + dir_opendss)


##############################
## add nodes to Node kernel ##
##############################
# Node kernel uses kernel_api to get a list of node names from OpenDSS
# then create a dictionary of node_ids to track info on these nodes.
# https://github.com/lbnl-cybersecurity/ceds-cigar/blob/toan_dev/pycigar/core/kernel/node/opendss.py#L19
k_node.start_nodes()    
# check if we start the Node kernel successfully
print("List of nodes in the grid: {}".format(list(k_node.nodes.keys())))


#############################
## start the Device kernel ##
#############################
k_device.start_device()
# add a device to a node #
# Let's add a new device with device_id is pv_1, connect to the node s701a
# the type of device is PVDevice
# the controller of this device is AdaptiveInverterController
# no hack
from pycigar.devices import PVDevice
from pycigar.devices import StorageDevice
from pycigar.controllers import FixedController

adv_id = k_device.add(name='pv_1', connect_to='s701a', device=(PVDevice, {}), controller=(FixedController, {'default_control_setting':[1.00, 1.01, 1.01, 1.02, 1.04]}), adversary_controller=None, hack=None)
# the k_device.add() always return an adversarial id because of code convention
# this adv_id will be used later to keep track the dictionary of hack time 
# PyCIGAR always add a adversarial device beside the device you add, if there is no hack, the percentage is 0

adv_id = k_device.add(name='sd_1', connect_to='s701a', device=(StorageDevice, {}), controller=(FixedController, {'default_control_setting':[1.00, 1.01, 1.01, 1.02, 1.04]}), adversary_controller=None, hack=None)

# check if we add a device successfully
print("List of devices in the grid: {}".format(list(k_device.devices.keys())))


###########################################################
## add load-solar profiles to nodes with Scenario kernel ##
###########################################################
# k_scenario.change_load_profile() will add the load profiles to nodes
# k_scenario.change_load_profile() will distribute solar profiles to devices according to it percentage control
# https://github.com/lbnl-cybersecurity/ceds-cigar/blob/toan_dev/pycigar/core/kernel/scenario/opendss.py#L210
dir_load_solar = "./data/load_solar_data.csv"
k_scenario.change_load_profile(start_time=100, end_time=500, network_data_directory_path=dir_load_solar)

# check if load profiles are loaded to nodes
sample_node = k_node.get_node_ids()[0] # get a sample node
print("Load profile at node {}: {}".format(sample_node, k_node.nodes[sample_node]["load"][:3]))

# check if solar profiles are loaded to devices
sample_device = k_device.get_pv_device_ids()[0]
print("Solar profile at device {}: {}".format(sample_device, k_device.devices[sample_device]["device"].solar_generation[:3]))

print('Complete')

List of nodes in the grid: ['s701a', 's702a', 's703a']
List of devices in the grid: ['pv_1', 'adversary_pv_1', 'sd_1', 'adversary_sd_1']
Load profile at node s701a: [233.67400227 233.5913791  233.61601926]
Solar profile at device pv_1: [96.94761022 97.03513774 97.1306978 ]
Complete


In [4]:
x = deque([0, 1], maxlen=2)
x.append(2)
x

deque([1, 2])